In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
class RNNRegressor:
    def __init__(self, radius=1.0, weights='uniform'):
        self.radius = radius
        self.weights=weights
        self.X = None
        self.y = None

    def fit(self, X_train, y_train):
        self.X = np.asarray(X_train)
        self.y = np.asarray(y_train)

    def predict(self, X_test):
        X_test = np.asarray(X_test)
        test_predictions = []

        for test_sample in range(X_test.shape[0]):
            neighbor_distances = []
            indices = []
            for train_sample in range(self.X.shape[0]):
                dist = np.linalg.norm(X_test[test_sample] - self.X[train_sample])
                if dist <= self.radius:
                    neighbor_distances.append(dist)
                    indices.append(train_sample)

            if indices:
                neighbors = [self.y[i] for i in indices]
                if self.weights == 'distance':
                    if 0.0 in neighbor_distances:
                        pred = self.y[indices[neighbor_distances.index(0.0)]]
                    else:
                        weights = [1/(d+1e-5) for d in neighbor_distances]
                        weighted_sum = np.dot(weights, neighbors)
                        weights_total = np.sum(weights)
                        pred = weighted_sum /  weights_total
                else:
                    pred = np.mean(neighbors)
            else:
                # No neighbors found; fallback to global mean
                pred = np.mean(self.y)

            test_predictions.append(pred)
        return np.asarray(test_predictions)

    def score(self, y_pred, y_test):
        y_pred = np.asarray(y_pred)
        y_test = np.asarray(y_test)
        u = ((y_test - y_pred) ** 2).sum()
        v = ((y_test - y_test.mean()) ** 2).sum()
        r2 = 1 - u / v
        return r2
        

In [3]:
import numpy as np

X_train = np.array([[0], [1], [2], [3], [4]])
y_train = np.array([0, 1, 2, 3, 4])

X_test = np.array([[1.5], [3.5], [10]]) 
model = RNNRegressor(radius=1.0, weights='distance')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_test = np.array([1.5, 3.5, 2.0])  # Pretend true labels (for illustration)

# Output predictions and score
print("Predictions:", y_pred)
print("R² Score:", model.score(y_pred, y_test))


Predictions: [1.5 3.5 2. ]
R² Score: 1.0


In [4]:
import numpy as np
from collections import Counter

class RNNClassifier:
    def __init__(self, radius=1.0, weights='uniform'):
        self.radius = radius
        self.weights = weights
        self.X = None
        self.y = None

    def fit(self, X_train, y_train):
        self.X = np.asarray(X_train)
        self.y = np.asarray(y_train)

    def predict(self, X_test):
        X_test = np.asarray(X_test)
        test_predictions = []

        for test_sample in range(X_test.shape[0]):
            neighbor_distances = []
            indices = []
            for train_sample in range(self.X.shape[0]):
                dist = np.linalg.norm(X_test[test_sample] - self.X[train_sample])
                if dist <= self.radius:
                    neighbor_distances.append(dist)
                    indices.append(train_sample)

            if indices:
                neighbors = [self.y[i] for i in indices]
                if self.weights == 'distance':
                    if any(np.isclose(d, 0.0) for d in neighbor_distances):
                        zero_idx = next(i for i, d in enumerate(neighbor_distances) if np.isclose(d, 0.0))
                        pred = neighbors[zero_idx]
                    else:
                        weights = [1 / (d + 1e-5) for d in neighbor_distances]
                        class_scores = {}
                        for label, weight in zip(neighbors, weights):
                            class_scores[label] = class_scores.get(label, 0) + weight
                        pred = max(class_scores, key=class_scores.get)
                else:
                    pred = Counter(neighbors).most_common(1)[0][0]
            else:
                # No neighbors found; fallback to global mode
                pred = Counter(self.y).most_common(1)[0][0]

            test_predictions.append(pred)
        return np.asarray(test_predictions)

    def score(self, y_pred, y_true):
        y_pred = np.asarray(y_pred)
        y_true = np.asarray(y_true)
        return (y_pred == y_true).mean()


In [6]:
# Sample classification data
X_train = np.array([[0], [1], [2], [3], [4]])
y_train = np.array([0, 0, 1, 1, 1])

X_test = np.array([[1.5], [3.5], [10]])  # 10 has no close neighbors

# Initialize and train
model = RNNClassifier(radius=1.0, weights='distance')
model.fit(X_train, y_train)

# Predict and score
y_pred = model.predict(X_test)
y_test = np.array([0, 1, 1])  # Hypothetical true labels

print("Predictions:", y_pred)
print("Accuracy:", model.score(y_pred, y_test))


Predictions: [0 1 1]
Accuracy: 1.0
